# GPT-4 API: Extract Structured Data from SEC Filings (Material Event Disclosures)

Data sources used:
- SEC-API.io
- AlgoSeek
- WRDS Fundamental Data

Logic:

- Query API to download metadata of 8-Ks with Item 4.02 => create file `8K-filing-metadata.csv`
- Extractor API to extract and download Item 4.02 sections => save to `./data/edgar/8K-item-4.02/CIK/ACCESSION-NO-item-4-2.txt`
- Pre-select 100 filings that deal with revenue recognition errors, and where 50% impact is material and 50% not-material.
- For each item section, extract structured data using GPT4 and save to `./data/edgar/8K-item-4.02/CIK/ACCESSION-NO-item-4-2-structured-data.json`
- For each CIK, and `filedAt`, read `Years since IPO` and `market cap` from `fundamentals`
- For each filing, calculate 1/2/3/5/10/20 day return after filing was disclosed
  - Plot return distributions for each day
  - Display descriptive stats

## End Result

![Return Histogram](./assets/returns-histogram-4.02.png)

![Descp Stats](./assets/desc-stats-returns-4.02.png)